<b>D.</b>
Составить программу, которая используя API к одной из поисковых систем, например:
* Сервис Яндекс.XML, позволяющий отправлять запросы к поисковой базе Яндекса и получать ответы в формате XML (https://tech.yandex.ru/xml/)
* Google Custom Search (https://developers.google.com/custom-search/)
получает тексты ответов (сниппетов) на заданный запрос (или же извлекает сами текстовые документы) и разбивает множество полученных текстов на кластеры.  

Рассмотреть несколько (3-5) различных запросов и соответствующую выдачу поисковиком 25-90 текстов, проанализировать результаты кластеризации.

<b>Отчет</b>: описание составленной программы (ее текст с комментариями), используемого API и метода кластеризации (включая меру близости текстов), примеры запросов к поисковику, характеристика полученных кластеров и выводы по анализу результатов.

* Мы будем использовать Google Custom Search API, а точнее Custom Search API для Python.
* Мы будем использовать настраиваемую Custom Search Engine (нам будет необходим CSE ID) - именно в ней мы пропишем настройки для поиска по всему интернету, языка поиска, для фильтрации дублирующихся документов и проч.

Ниже следует краткая инструкция со ссылками по подключению CS API, установке CS API for Python и настройке CSE и их совместному использованию.

* create a Google Account
* create a Billing Account on Google Cloud Platform

https://developers.google.com/api-client-library/python/start/get_started

https://developers.google.com/api-client-library/python/auth/api-keys

Custom Search API for Python
* create a project in the Google API Console (https://console.developers.google.com/)
* create credentials to access your enabled API  
(get Google API key)
* enable Custom Search API
* link Billing Account

https://stackoverflow.com/questions/37083058/programmatically-searching-google-in-python-using-custom-search

Custom Search Engine for Python
* setup Custom Search Engine (https://cse.google.com/cse)  
(select Search the entire web)  
(get Custom Search Engine ID)  
(set Search Language and others Settings)

https://developers.google.com/api-client-library/python/start/installation

Google API client for Python
*  install Google API client for Python (https://github.com/google/google-api-python-client)  
(pip install --upgrade google-api-python-client)

PyDoc reference for the CustomSearch API:  
https://developers.google.com/resources/api-libraries/documentation/customsearch/v1/python/latest/

my_api_key = <em>Custom Search API for Python</em> key  
my_cse_id = <em>Custom Search Engine for Python</em> id

In [133]:
# googleapiclient.discovery - a client library for Google's discovery based APIs
import googleapiclient.discovery as GoogleAPIs

# Custom Search API for Python
api_key = "Google API key"
# function 'build' constructs a Resource object for interacting with an API
# (the serviceName and version are the names from the Discovery service)
service = GoogleAPIs.build(serviceName = 'customsearch',
                           version = 'v1',
                           developerKey = api_key)
# and service.cse() returns the cse Resource with 'list' method
help(service.cse().list)

# Custom Search Engine for Python
ru_cse_id = "Custom Search Engine ID" #cse language: russian
en_cse_id = "Custom Search Engine ID" #cse language: english

# Google Chrome User Agent for good requests
headers = {'user-agent':
           'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}

Help on method method in module googleapiclient.discovery:

method(**kwargs) method of googleapiclient.discovery.Resource instance
    Returns metadata about the search performed, metadata about the custom search engine used for the search, and the search results.
    
    Args:
      q: string, Query (required)
      gl: string, Geolocation of end user.
      siteSearch: string, Specifies all search results should be pages from a given site
      lowRange: string, Creates a range in form as_nlo value..as_nhi value and attempts to append it to query
      filter: string, Controls turning on or off the duplicate content filter.
        Allowed values
          0 - Turns off duplicate content filter.
          1 - Turns on duplicate content filter.
      start: integer, The index of the first result to return
      rights: string, Filters based on licensing. Supported values include: cc_publicdomain, cc_attribute, cc_sharealike, cc_noncommercial, cc_nonderived and combinations of these.


CustomSearch API . cse  
<b>list</b> Method details:  
https://developers.google.com/resources/api-libraries/documentation/customsearch/v1/python/latest/customsearch_v1.cse.html

Согласно заданию, нас интересуют текстовые запросы и тексты ответов (или же сами текстовые документы):  
* необходимыми параметрами запроса является текст запроса и Custom Search Engine ID; также включим 'duplicate content filter'
* интересующими нас параметрами ответа мы будем считать:  
    * 'snippet' (сниппет "документа") и 'link' (прямую ссылка на "документ"), как наиболее информативные;  
    * 'title' (заголовок "документа") и 'displayLink' (ссылка на сайт, где размещен "документ");
    * 'fileFormat' (формат "документа", если определен)

Под "документом" понимается текстовое представление найденной страницы сайта или же документ в обычном понимании этого слова.  
* помимо сниппета, краткой текстовой характеристики документа, будем формировать сам текст самого документа: 'document_text', используя BeautifulSoup. Это дорогостоящая операция, поэтому будем выполнять ее в зависимости от параметра 'text_type', отвечающего за желаемое представление текста (в виде сниппета или в виде текстового представления страницы).

В качестве страницы, чье текстовое представление мы хотим получить, мы можем рассматривать либо прямую ссылку, либо формировать ссылку на кэшированную страницу гугла (тем более, что гуглом уже сделано ее текстовое представление). При рассматривании прямой ссылки необходимо помнить о том, что некоторые сайты выкидывают bad request (так, например, делает кинопоиск): для этого в качестве 'user_agent' мы будем передавать корректный User Agent нашего браузера.

Функция google_search будет возвращать требуемые нами параметры ответов на запрос, num_google_search - возвращать полученные при помощи google_search результаты, чье количество равно 'num'. Продемонстрируем работу этой функции, выведя для первых пяти результатов по запросу "замок" их 'title', 'link', 'snippet', а также первые 100 символов 'text' (что не очень информативно, к сожалению, но показывает наличие большого количества служебных терминов разметки html в полученных текстах).

In [169]:
import requests
from bs4 import BeautifulSoup
import re

def document_text(items, text_type):
    if text_type == 'snippet':
        return items['snippet']
    else:
        link = items['link']
        # if 'cacheId' in items:
        #    link = 'https://webcache.googleusercontent.com/search?q=cache:{}:{}&strip=1'.format(items['cacheId'], items['link'])
        cache_text = requests.get(link, headers = headers).text
        return ' '.join(BeautifulSoup(cache_text, 'lxml').findAll(text=True))

def google_search(search_term, cse_id, text_type, **kwargs):
    res = service.cse().list(q = search_term,
                             cx = cse_id,
                             **kwargs).execute()
    required_keys = ['snippet', 'title', 'link', 'cacheId', 'displayLink', 'fileFormat']
    return [dict([(key, items[key]) for key in required_keys if key in items] +
                 [('text', document_text(items, text_type))]) for items in res['items']]

def num_google_search(search_term, num = 10, text_type = 'snippet', cse_id = ru_cse_id, **kwargs):
    results = []
    if num%10 > 0:
        results += google_search(search_term, cse_id, text_type, start = 1, num = num%10, **kwargs)
    for i in range(num//10):
        results += google_search(search_term, cse_id, text_type, start = 10 * i + num%10 + 1, num = 10, **kwargs)
    return results

results = num_google_search('замок', num = 50, text_type = 'text')
for i in range(5):
    print('{}. {}'.format(i, results[i]['title']))
    for key in ['link', 'cacheId', 'snippet']:
        print('{} : {}'.format(key, results[i][key]))
    print('{}...\n'.format(results[i]['text'][:100]))

0. Замок (строение) — Википедия
link : https://ru.wikipedia.org/wiki/%D0%97%D0%B0%D0%BC%D0%BE%D0%BA_(%D1%81%D1%82%D1%80%D0%BE%D0%B5%D0%BD%D0%B8%D0%B5)
cacheId : COQ1MknSg4oJ
snippet : За́мок (от польск. zamek через чеш. zámek, калькированного с ср.-в.-нем.slōʒ 
— «за́мок» и «замо́к». Средневерхненемецкое слово калькирует лат. clusa ...
html 
 
 
 Замок (строение) — Википедия 
 document.documentElement.className = document.documentElem...

1. СУПЕР ЗАМОК ПРИНЦЕССЫ Большой Игрушечный Домик Игры ...
link : https://www.youtube.com/watch?v=AXUcFtbc658
cacheId : vTfdgltjSCkJ
snippet : 28 авг 2016 ... Новый Замок Принцессы - настоящая Сказка! Это Самый большой и Самый 
Красивый Замок который мы выдели. Большой Замок для ...
html  Origin Trial Token, feature = Long Task Observer, origin = https://www.youtube.com, expires = ...

2. Замок (устройство) — Википедия
link : https://ru.wikipedia.org/wiki/%D0%97%D0%B0%D0%BC%D0%BE%D0%BA_(%D1%83%D1%81%D1%82%D1%80%D0%BE%D0%B9%D1%81%D1%82%D0%B2%D0%BE)
cac

Представим каждый документ, как список терминов, которые в него входят (с повторениями).  
В дальнейшем нам потребуется выяснять сходство пар текстов, то есть вычислять некоторое "расстояние" между ними (об этом подробнее дальше), при этом это сходство будет опираться на термины, которые в этих текстах присутствуют. Веса tf-idf учитывают меньшую информативность частотных терминов; но наше текстовое представление документа обладает рядом недостатков - например, в него попадают служебные термины. Можно предпринять некоторые меры по устранению этого недостатка: например, рассматривать текстовое представление кэшированной страницы, которое было сделано гуглом, - это даст нам основания полагать, что html-разметка будет как минимум упрощена, а в лучшем случае - схожа для всей коллекции. Как минимум, постараемся ограничить находимые нами термины следующими условиями: во-первых, приведем их к нижнему регистру, во-вторых, за термины будем считать только буквенные последовательности (возможно, с подчеркиваниями) - за разделители между словами будем считать только пробельные символы, а все прочие удалим.

Далее мы хотим представить каждый документ вектором весов $\text{tf-idf}$ всех терминов. Для каждого документа вычислим <em>tf-idf</em> терминов, в него входящих (для всех остальных $\text{tf-idf}$ равен 0):
$$\text{tf-idf} = tf_{t, d} \cdot idf_t = tf_{t, d} \cdot \log{\frac{N}{df_t}} = tf_{t, d} \cdot (\log{N} - \log{df_t}), \text{где}$$
* $tf_{t, d}$ - частота термина $t$ в документе $d$, т.е. количество входждений термина $t$ в документ $d$;  
* $df_t$ - подокументная частнотность термина $t$, т.е. количество документов коллекции, содержащих термин $t$;  
* $idf_t$ - обратная $df_t$ величина, равная $\log{\frac{N}{df_t}}$, где $Ν$ - число документов коллекции.

Таким образом, $tf_{t, d} = 0 \Rightarrow \text{tf-idf} = 0$. Для вычисления веса $\text{tf-idf}$ каждого термина каждого документа создадим инвертированный список Термин-Документ, в котором для каждого термина $t$, встречающегося в коллекции будем хранить индексы документов $d$, в которые он входит, и соответствующие этим индексам $tf_{t, d}$, т.е. словарь вида $\text{term_doc} = \{t : \{d : tf_{t, d}\}\}$.  
Тогда для каждого термина $t$ документа с индексом $d$
$$\text{tf-idf}_{t, d} = \text{term_doc}[t][d] \cdot (\log{\text{len}_{\text{docs}}} - \log{\text{len}_{\text{term_doc}[t]}})$$
Обратим внимание, что $\text{term_doc}[t][d]$ определен только для терминов, входящих в этот документ; в ином случае необходимо вернуть 0.

Функция term_vectors возвращает полученный инвертированный список Термин-Документ term_doc и представление документов в векторной модели (т.е. каждый документ представляется в виде мешка слов с их весами tf-idf; для каждого документа хранятся термины с ненулевыми весами).  
Продемонстрируем работу этой функции, выведя для первых 5 документов 5 наиболее и 5 наименее весомых терминов.

In [225]:
from math import log

def term_vectors(results):
    # documents' representation: lists of termins
    docs = [re.sub(r'[^ \w]|[\d]', '', res['text'].lower()).split(' ') for res in results]
    # inverted term-document list
    term_doc = {}
    for i in range(len(docs)):
        for term in docs[i]:
            if term in term_doc:
                if i in term_doc[term]:
                    term_doc[term][i] += 1
                else:
                    term_doc[term][i] = 1
            else:
                term_doc[term] = {i : 1}

    # term frequency
    term_freq = lambda term, doc_id: term_doc[term][doc_id] if doc_id in term_doc[term] else 0
    # document frequency
    doc_freq = lambda term: len(term_doc[term])
    # inverse document frequency
    inv_doc_freq = lambda term: log(len(docs)) - log(doc_freq(term))
    
    # tf-idf vector model (bag-of-words)
    bag_of_words = [{t : term_freq(t, i) * inv_doc_freq(t) for t in docs[i]} for i in range(len(docs))]

    return term_doc, bag_of_words

term_doc, bag_of_words = term_vectors(results)
for i in range(5):
    print('{}. {}'.format(i, results[i]['title']))
    keys = sorted(bag_of_words[i].items(), key = lambda x: x[1], reverse = True)
    for j in range(5):
        print('{} : {}'.format(*keys[j]))
    print('...')
    for j in range(6, 1, -1):
        print('{} : {}'.format(*keys[-j]))
    print()

0. Замок (строение) — Википедия
править : 67.07446871508684
викитекст : 64.70844648548083
замки : 44.240607024844515
замков : 43.981955129959445
ров : 32.18875824868201
...
ie : 0.41551544396166573
script : 0.2231435513142097
if : 0.10536051565782634
function : 0.08338160893905089
html : 0.020202707317519497

1. СУПЕР ЗАМОК ПРИНЦЕССЫ Большой Игрушечный Домик Игры ...
плейлист : 297.3137484125391
princess : 230.8093573202606
kids : 186.69479784235563
diana : 183.86508125512287
принцессы : 177.03817036775104
...
вы : 0.3856624808119844
из : 0.3487067742895551
s : 0.32850406697203605
от : 0.24846135929849966
html : 0.020202707317519497

2. Замок (устройство) — Википедия
замки : 55.65753786996568
засов : 50.856299070565896
кг : 45.06426154815481
г : 40.88144878100393
замков : 40.31679220246282
...
за : 0.3011050927839216
script : 0.2231435513142097
if : 0.10536051565782634
function : 0.08338160893905089
html : 0.020202707317519497

3. Торговый центр Замок
ð : 25.328436022934504
ñðµð½ññ : 1

Еще раз отметим тот факт, что полученное нами представление документов в виде bag_of_words не является векторным в обычном понимании этого слова: мы не храним нулевые значения. Таким образом, это необходимо учитывать при вычислении косинусной меры сходства двух таких представлений $q$ и $d$:
$$cos(q, d) = \dfrac{\sum{q_t \cdot d_t}}{\sqrt{\sum{q_t^2}}\cdot\sqrt{\sum{d_t^2}}},$$
при этом:
* при суммировании в числителе не зануляются только те члены, что соответствуют терминам, принадлежащим и $q$, и $d$,  
следовательно, суммирование можно проводить не по всем терминам, а по пересечению терминов этих двух документов;
* при суммированиях в знаменателе не зануляются только те члены, что соответствуют терминам, принадлежащим или $q$, или $d$ соответственно,  
следовательно, суммирования можно проводить не по всем терминам, а по терминам соответствующего документа.

Покажем для первых 10 результатов их попарное сравнение и отметим некоторые вполне ожидаемые закономерности:
* матрица получается симметричной - сходство двух текстов не зависит от их порядка;
* диагональ матрицы получается единичной - сходство текста с самим собой максимально;
* члены матрицы - числа от 0 до 1, поскольку веса являются неотрицательными величинами, а значит скалярное произведение будет положительным; косинус же определен на интервале $[-1; 1]$.

In [227]:
import pandas as pd

# cosine similarity measure
cos = lambda q, d: sum([q[t] * d[t] for t in set(q) & set(d)])/pow(sum([q[t] ** 2 for t in q]) *
                                                                   sum([d[t] ** 2 for t in d]), 0.5)
for i in range(10):
    print('{} : {}\n{}\n{}'.format(i, results[i]['title'], results[i]['displayLink'], results[i]['snippet']))
df = pd.DataFrame([[cos(q, d) for d in bag_of_words[:10]] for q in bag_of_words[:10]])
print(df)

0 : Замок (строение) — Википедия
ru.wikipedia.org
За́мок (от польск. zamek через чеш. zámek, калькированного с ср.-в.-нем.slōʒ 
— «за́мок» и «замо́к». Средневерхненемецкое слово калькирует лат. clusa ...
1 : СУПЕР ЗАМОК ПРИНЦЕССЫ Большой Игрушечный Домик Игры ...
www.youtube.com
28 авг 2016 ... Новый Замок Принцессы - настоящая Сказка! Это Самый большой и Самый 
Красивый Замок который мы выдели. Большой Замок для ...
2 : Замок (устройство) — Википедия
ru.wikipedia.org
Замо́к — механическое, электронное или комбинированное устройство 
фиксации. Применяется для запирания дверей, крышек, ёмкостей и пр., а 
также ...
3 : Торговый центр Замок
www.tczamok.by
Торговый центр Замок – новый уровень шопинга и развлечений! К вашим 
услугам - магазины, кафе и рестораны, ледовый каток, кинотеатр, детский ...
4 : Замок — Википедия
ru.wikipedia.org
За́мок. Замок (строение) — здание (или комплекс зданий), сочетающее в 
себе жилые и оборонительно-фортификационные задачи. В наиболее ...
5 : замок - Wikti

Поверхностно проанализируем полученные результаты: например, высоким (относительно прочих пар) сходством обладают тексты 0, 2 и 4; 7 и 9. И это действительно имеет смысл: 0, 2 и 4 тексты являются статьями в Википедии по запросу замок - при этом наша мера не позволила оценить разницу в значениях слова "замок" (статьи 0 и 2 являются семантически различными). Относительно 7 и 9 текстов все проще и интереснее: наше сходство позволило оценить близость текстов про замки в игровых локациях; нельзя не обратить внимание, что они принадлежат к группе Вики-сообществ (http://ru.community.wikia.com/wiki/Вики_Сообщества).

Ради интереса посмотрим на множества общих терминов документов 0, 2 и 4. Поскольку мы не выбрасывали служебные термины и термины, не имеющие прямого отношения к запросу, то они оказали большое влияние на сходство этих документов.

In [239]:
print(set(bag_of_words[0]) & set(bag_of_words[2]) & set(bag_of_words[4]))

{'', 'size', 'ссылки', 'книгу', 'usage', 'целью', 'перейти', 'замок', 'if', 'ссылками', 'других', 'сообщество', 'википедии', 'by', 'если', 'pdf', 'key', 'windowrlqwindowrlqpushfunctionmwconfigsetwgbackendresponsetimewghostnamemw', 'имён', 'цитировать', 'элемент', 'некоммерческой', 'к', 'история', 'указатель', 'secondsreal', 'новые', 'content', 'правки', 'inc', 'module', 'для', 'печатьэкспорт', 'использования', 'постоянная', 'отдельных', 'expansion', 'справка', 'европе', 'рубрикация', 'навигация', 'report', 'revision', 'with', 'участие', 'обошибке', 'могут', 'с', 'wikimedia', 'доступен', 'так', 'события', 'argument', 'postexpand', 'без', 'отказ', 'дополнительные', 'спецстраницы', 'expensive', 'википедия', 'зарегистрированный', 'windowrlqwindowrlqpushfunctionmwconfigsetwgpageparsereportlimitreportcputimewalltimeppvisitednodesvaluelimitppgeneratednodesvaluelimitpostexpandincludesizevaluelimittemplateargumentsizevaluelimitexpansiondepthvaluelimitexpensivefunctioncountvaluelimitentityaccess

Проведем кластеризацию полученной коллекции документов с вычисленными весами терминов. Для этого воспользуемся методом k средних, который реализован в sklearn.cluster. Отметим следующую особенность данного метода: во-первых, число кластеров должно быть изначально задано (по умолчанию n_clusters равно 8); во-вторых, расстояние между векторами вычисляется по евклидовой метрике. Если быть точнее, в начале инициализируется n центроидов, а затем минимизируется суммарное квадратичное отклонение точек кластеров от центроидов; затем центроиды пересчитываются, как среднее точек соответствующих кластеров, и итерация повторяется. Поскольку изначальное инициализирование центроидов необходимо для дальнейшей работы, то метод проводится несколько раз (по умолчанию n_init равно 10), а затем выбирается наилучший результат.

Итак, есть два момента, которые следует обговорить: первый заключается в том, что наша метрика не является евклидовой; второй - центроиды не представляются в виде вектора, не совпадающего с векторными представлениями наших документов.  
* Во-первых, заметим, что при нормализации весовых векторов, соответствующих документам, косинусное сходство представляет собой скалярное произведение этих векторов: $\sum{q_t \cdot d_t}$; квадратичное же отклонение между ними равно $\sum{(q_t - d_t)^2} = \sum{q_t^2 + d_t^2 - 2q_t\cdot d_t} = 2 - 2 \sum{q_t \cdot d_t}$, поскольку вектора были нормализованы, а значит они лежат на единичной $Τ$-мерной сфере, где $Τ$ - число различных терминов коллекции. Таким образом, близость по этой метрике между векторами на единичной сфере соответствует максимуму скалярного произведения, то есть близости по косинусной мере;
* Во-вторых, заметим, что центроиды не будут лежать на единичной сфере, поскольку являются средним точек, входящих в кластер; при этом расстояние между точками кластера будет меньше, чем сумма расстояний от этих точек до центроида, а значит при минимизации суммы квадратичных отклонений точек кластеров от центроидов минимизируется и расстояние между этими точками по косинусной мере.

In [271]:
from scipy.sparse import lil_matrix
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans

def clusterisation(term_doc, bag_of_words, results, n_clusters, n_init):
    # weight matrix
    weight_matrix = lil_matrix((len(term_doc), len(bag_of_words)))
    terms = list(term_doc)
    for t in range(len(terms)):
        for d in range(len(bag_of_words)):
            if terms[t] in bag_of_words[d]:
                weight_matrix[t, d] = bag_of_words[d][terms[t]]

    # doc-vector's normalization
    X = normalize(weight_matrix.T)
    kmeans = KMeans(n_clusters = n_clusters, n_init = n_init).fit(X)
    
    clusters = {}
    for i in range(len(results)):
        label, title, display_link, snippet = (kmeans.labels_[i], results[i]['title'],
                                               results[i]['displayLink'], results[i]['snippet'])
        if label not in clusters:
            clusters[label] = [(title, display_link, snippet)]
        else:
            clusters[label].append((title, display_link, snippet))
    return sorted(clusters.items(), key = lambda x: len(x[1]), reverse = True)

In [259]:
clusters = clusterisation(term_doc, bag_of_words, results, n_clusters = 20, n_init = 100)
for c in clusters:
    for i in c[1]:
        print("* {}\n{}\n{}".format(*i))
    print('\n=====================\n')

* Замок | Убежище | FANDOM powered by Wikia
ru.fallout.wikia.com
Замок (англ. The Castle) — локация Fallout 4. Бывшая/текущая штаб-квартира 
Минитменов в…
* Замок Кейнхерст | Bloodborne вики | FANDOM powered by Wikia
ru.bloodborne.wikia.com
Замок Кейнхерст (англ. Cainhurst Castle) - локация в игре Bloodborne. 
Огромный замок Кейнхерст…
* Замок Крастера | Игра Престолов Вики | FANDOM powered by Wikia
ru.gameofthrones.wikia.com
Замок Крастера — название, данное укреплённому дому, расположенному 
за Стеной. В поместье жил…
* Замок Дранглик | Dark Souls вики | FANDOM powered by Wikia
ru.darksouls.wikia.com
Замок Дранглик (англ. Drangleic Castle) - локация в игре Dark Souls II. По 
возращении из-за моря…
* Самоцветный замок | Террария вики | FANDOM powered by Wikia
ru.terraria.wikia.com
Самоцветный замок (Gem Lock) — новый механизм, добавленный в 
обновлении 1.3.1. Если в замок вставить большой самоцвет, то замок начнет
 ...
* Замок Ла Валетт | Ведьмак Вики | FANDOM powered by Wikia
vedmak.

Итак, мы можем отметить, что кластеры, состоящие из наибольшого количество элементов, наиболее корректно сформированные: например, в один кластер попали ответы, соответствующие описаниям замков в игровых локациях (FANDOM powered by Wikia); в другой кластер попадают статьи в википедии (как разбор слова, так и описание понятия); еще один кластер соответствует описаниям реально существующих замков и замков-заповедников; еще один - описаниям развлекательных центров и фильмов. Разумеется, есть выбросы в кластерах с меньшим количеством элементов, но тема угадывается, что дает надежду полагать, что кластеризация работает.

Все же стоит отметить, что k-means - не самый точный метод кластеризации.

Рассмотрим еще несколько примеров запросов:

In [272]:
results = num_google_search('замок', 50, 'snippet')
for c in clusterisation(*term_vectors(results), results, 20, 100):
    for i in c[1]:
        print("* {}\n{}\n{}".format(*i))
    print('\n=====================\n')

* Сказочный замок - играйте в онлайн игру шарики бесплатно
www.ishariki.ru
Перед вами круглый Сказочный Замок. По его стене катается маленькая 
пушка, но нацелена она внутрь замка! А на главной площади – несметное ...
* Единственный дракон. Замок княгини читать книгу онлайн на Lit ...
lit-era.com
Замок княгини, жанр: Приключенческое фэнтези, автор Наталья Сапункова. 
Читайте Единственный дракон. Замок княгини на сайте Самиздат Lit-Era.
* Михайловский замок
www.rusmuseum.ru
Михайловский замок. На информационной стойке вы можете узнать о 
режиме работы музея, временных выставках, мероприятиях, лекциях, 
занятиях и ...
* Замок Локет
www.hradloket.cz
Как в замок. К самому замку Вы можете добраться пешком и верхом или на 
велосипеде, на лодке по реке, но, конечно, и на машине или поезде.
* Вози замок с собой: как надёжно запереть дверь в отеле ...
www.popmech.ru
10 окт 2017 ... Если вы когда-либо жили в отеле или, скажем, снимали комнату в квартире, 
где живут другие люди, вы знаете эту про

Как можно видеть, одних сниппетов недостаточно для корректной кластеризации; хотя один из кластеров и соответствует FANDOM powered by Wikia.

Попробуем другой короткий запрос - "игры" (к тому же он более конкретный - у слова "замок" есть две очевидных различных интерпретации).

In [281]:
results = num_google_search('игры', 50, 'text')
for c in clusterisation(*term_vectors(results), results, 15, 50):
    for i in c[1]:
        print("* {}\n{}\n{}".format(*i))
    print('\n=====================\n')

* CIT: российские военные выдали кадры из игры за атаку на ИГ ...
www.bbc.com
1 день назад ... Минобороны России выдало кадры из мобильной игры за аэросъемку 
боевой операции против экстремистской группировки ИГ в Сирии ...
* Интернет-магазин Chrome - Игры
chrome.google.com
Тысячи игр, в которые можно играть через браузер: аркады и экшн, ролевые 
игры и стратегии, спортивные, настольные и карточные игры.
* Игра — Википедия
ru.wikipedia.org
Игра́ — тип осмысленной непродуктивной деятельности, где мотив лежит не 
в ее результате, а в самом процессе. Также термин «игра» используют для ...
* ″Неоспоримые″ доказательства Минобороны РФ оказались ...
www.dw.com
1 день назад ... Россия. "Неоспоримые" доказательства Минобороны РФ оказались 
скриншотом из игры. Когда пользователи соцсетей доказали, что ...
* Path of Exile
ru.pathofexile.com
Path of Exile - это сетевая ролевая игра активного действия в мрачном 
колдовском мире Рэкласта. В основу игры легли: мощная бартерная 
экономика, ...
* Укло

Сложно проанализировать точно, почему первый кластер был сформирован таким образом: можем видеть, что документы про миноборону РФ попали в один кластер; "Игры-Онлайн" попали во второй, а "FANDOM powered by Wikia", аналогично запросу по "замок", также попали в один кластер; то же произошло и с играми на "Steam".

In [283]:
results = num_google_search('стена замок тошнота чума', 25, 'text')
for c in clusterisation(*term_vectors(results), results, 10, 50):
    for i in c[1]:
        print("* {}\n{}\n{}".format(*i))
    print('\n=====================\n')

* О гигиене в средневековой Европе | Фактор Времени
faktorvremeny.wordpress.com
7 апр 2013 ... В результате этого перекоса в сторону молитв, от чумы погибло чуть-ли .... 
двор периодически переезжал из замка в замок из-за того, что в ... Стены 
замков оборудовались тяжелыми портьерами, ...... Поскольку внешний вид 
трупа неотвратно вызывает позывы тошноты, хорошо бы месяц ...
* Альбер Камю. Бунтующий человек
lib.ru
Иногда персонажи выступают как alter ego автора: так, в "Чуме" все 
основные ... на "Тошноту" он отмечает схематизм, известную ходульность 
образов. ..... собой "философское самоубийство" -- "скачок" через "стены 
абсурда".
* Шишова Зинаида Константиновна. Джек-соломинка
lib.ru
Чума пришла из Китая в Константинополь, в Египет, в Крым и на 
Средиземное море. ... В королевском замке - в Тауэре - было 1200 хорошо 
вооруженных рыцарей, .... Лоренса. Дети смотрели, как упрямо бросается 
пламя на стену, и в их глазах ... Джоанна от страха почувствовала слабость и 
тошноту.
* Генри

Первый кластер соответствует различным книгам, соответствующим запросам (к сожалению, с выбросами). Третий же объединен одной display Link.